### 1. 실습을 위해서 기존에 다운로드 받은 실습 파일을 바탕화면 mongo폴더에 압축을 해제합니다.
- 압축을 해제하면 쥬피터 노트북 파일(csv_to_mongo.ipynb) 및 데이터 파일(wc_match.tsv, team.tsv, players.tsv)가 포함되어 있습니다.
- 압축 파일을 바탕화면에 만들어둔 mongo폴더에 해제합니다

### 2. 쥬피터 노트북 오른쪽 상단의 UPLOAD 버튼을 통해, csv_to_mongo.ipynb, wc_match.tsv, team.tsv, players.tsv 파일을 업로드 합니다.

### 3. csv_to_mongo.ipynb 파일 내 Mongo Connection을 위한 clinet 커넥션 값을 replace합니다
- Atlas -> connection -> connect your application -> standard connection string -> copy

### 4. 전체를 실행합니다.

# Pandas로 MongoDB의 Data 처리하기

In [1]:
from pymongo import MongoClient

In [2]:
# mongoDB에 접속할 client 생성

murl = ''

client = MongoClient(murl, 27017)

In [3]:
football_database = client.Football

In [4]:
match_collection = football_database.wc_match

In [5]:
match_collection.count_documents({})

64

## 국가별 득점 랭킹

### 국가별 득점 가져오기(PyMongo)

* **Data 구조 확인**
    - 경기 { 경기 일자, 홈팀 { 국가, 득점, 선수 { 선수기록들 } }, 원정팀 { 국가, 득점, 선수 { 선수기록들 } } }

In [6]:
list(match_collection.find({}))[0]

{'_id': ObjectId('5be40747827e3c5216357ba3'),
 'away_team': {'country': 'Saudi Arabia',
  'players': [{'DC': 4027.0,
    'DCIP': 1818.0,
    'DCNIP': 1252.0,
    'Long_PA': 25.0,
    'Long_PC': 10.0,
    'Medium_PA': 9.0,
    'Medium_PC': 9.0,
    'SPRINT': 1.0,
    'Short_PA': 0.0,
    'Short_PC': 0.0,
    'TS': 14.83,
    'TTP': 5808.0,
    'Total_PA': 34.0,
    'Total_PC': 19.0,
    'Total_PP': 0.56,
    'club': 'Al Hilal (KSA)',
    'name': 'ABDULLAH ALMUAIOUF',
    'num': 1,
    'position': 'GK'},
   {'DC': 7997.0,
    'DCIP': 2836.0,
    'DCNIP': 3301.0,
    'Long_PA': 8.0,
    'Long_PC': 7.0,
    'Medium_PA': 39.0,
    'Medium_PC': 36.0,
    'SPRINT': 30.0,
    'Short_PA': 10.0,
    'Short_PC': 9.0,
    'TA3RD': 0.05,
    'TOH': 0.08,
    'TPA': 0.02,
    'TS': 27.9,
    'TTP': 5808.0,
    'Total_PA': 57.0,
    'Total_PC': 52.0,
    'Total_PP': 0.91,
    'club': 'Al Hilal (KSA)',
    'name': 'OSAMA HAWSAWI',
    'num': 3,
    'position': 'DF'},
   {'DC': 9040.0,
    'DCIP': 3494

* 원하는 Data 가져오기
    - Aggregate(Group By)를 이용하여 국가별 홈경기 총 득점, 원정경기 총 득점 Load

In [7]:
# pymongo를 이용한 MongoDB 결과값은 Cursor의 형태
# Cursor는 일종의 휘발성 변수로 1회 호출 후 바로 Memory에서 삭제
# 반드시 List, DataFrame 등으로 전환해야 지속적으로 사용 가능

home_goals = list(match_collection.aggregate([
    {'$group': {
        '_id': '$home_team.country',
        'home_goal': {'$sum' : '$home_team.score'}
    }},
    {'$sort': {'home_goal': -1}}
]))

away_goals = list(match_collection.aggregate([
    {'$group': {
        '_id': '$away_team.country',
        'away_goal': {'$sum' : '$away_team.score'}
    }},
    {'$sort': {'away_goal': -1}}
]))

* MongoDB에는 Join이 없기 때문의 위의 두 결과(국가별 홈경기 총득점과 원정경기 총득점)의 합을 구하기가 어려움
* MongoDB에서 복잡하지 않게 가능한 수준의 Filtering, Aggregating 된 결과를 Pandas를 통해 분석 수행하는 것이 보다 용이

### 국가별 득점 합치기

In [8]:
import pandas as pd

home_goal_pd = pd.DataFrame(home_goals)
away_goal_pd = pd.DataFrame(away_goals)

In [9]:
home_goal_pd.head()

,_id,home_goal
0,Belgium,13
1,France,12
2,Russia,10
3,England,6
4,Brazil,6


In [10]:
away_goal_pd.head()

,_id,away_goal
0,Croatia,9
1,England,6
2,Argentina,5
3,Colombia,4
4,Tunisia,4


* Pandas에서 2개 이상의 DataFrame을 하나로 합치는 방법은 3가지가 존재
    1. Merge
    2. Join
    3. Concatenate

##### 1. Merge

In [11]:
# 기본적으로는 동일한 Column 명을 Key로 Inner Join 수행
pd.merge(home_goal_pd, away_goal_pd)

,_id,home_goal,away_goal
0,Belgium,13,3
1,France,12,2
2,Russia,10,1
3,England,6,6
4,Brazil,6,2
5,Uruguay,6,1
6,Croatia,5,9
7,Portugal,4,2
8,Korea Republic,3,0
9,Nigeria,3,0


In [12]:
# 경우에 따라 parameter를 통해 Join 방식(how), Join Key(on) 지정 가능
pd.merge(home_goal_pd, away_goal_pd, how='inner', on='_id')

,_id,home_goal,away_goal
0,Belgium,13,3
1,France,12,2
2,Russia,10,1
3,England,6,6
4,Brazil,6,2
5,Uruguay,6,1
6,Croatia,5,9
7,Portugal,4,2
8,Korea Republic,3,0
9,Nigeria,3,0


##### 2. Join

In [13]:
# Join을 이용하여 동일한 결과 가능
# Index를 기준으로 하는 것이 보편적
# 단, Join이 수행되는 두 DataFrame에 동일한 Column 명이 존재하는 경우 suffix 지정이 필수
home_goal_pd.set_index('_id').join(away_goal_pd.set_index('_id'), lsuffix='_home', rsuffix='_away')

,home_goal,away_goal
_id,,
Belgium,13,3
France,12,2
Russia,10,1
England,6,6
Brazil,6,2
Uruguay,6,1
Croatia,5,9
Portugal,4,2
Korea Republic,3,0


### 홈경기 득점과 원정경기 득점을 병합하고 총 득점 계산

In [14]:
total_goals = pd.merge(home_goal_pd, away_goal_pd)
total_goals.head()

,_id,home_goal,away_goal
0,Belgium,13,3
1,France,12,2
2,Russia,10,1
3,England,6,6
4,Brazil,6,2


In [15]:
# total_goal = home_goal + away_gaol

total_goals['total_goals'] = total_goals.home_goal + total_goals.away_goal
total_goals.head()

,_id,home_goal,away_goal,total_goals
0,Belgium,13,3,16
1,France,12,2,14
2,Russia,10,1,11
3,England,6,6,12
4,Brazil,6,2,8


### 득점 순으로 정렬하기

In [16]:
# sort_values 함수는 DataFrame을 특정 Column의 값을 기준으로 정렬
# ascending Option은 기본은 True(오름차순)이나 False로 지정시 내림차순으로 정렬

total_goals.sort_values('total_goals', ascending=False)

,_id,home_goal,away_goal,total_goals
0,Belgium,13,3,16
6,Croatia,5,9,14
1,France,12,2,14
3,England,6,6,12
2,Russia,10,1,11
4,Brazil,6,2,8
5,Uruguay,6,1,7
10,Spain,3,4,7
12,Colombia,2,4,6
23,Argentina,1,5,6


## 그룹 연산

- 그룹핑(GROUP BY): db.COLLECTION_NAME.aggregate()
  - Aggregate는 match stage와 group stage, option으로 구분
  - match stage는 Collection에서 원하는 부분을 filter
    - ```$match : { COLUMN: VALUE }```
  - group stage는 Grouping key와 Grouping operation으로 구성
    - ```$group : { _id: GROUP KEY , GROUP FUNCTION : TARGET COLUMN}```
  - 각 stage를 list로 묶어서 aggregate의 인자로 전달

In [17]:
list(match_collection.aggregate([
    {'$match' : {'home_team.country' : {'$eq' : 'France'} } },
    {'$limit' : 1 }
]))

[{'_id': ObjectId('5be40747827e3c5216357ba7'),
  'away_team': {'country': 'Australia',
   'players': [{'DC': 4691.0,
     'DCIP': 1814.0,
     'DCNIP': 1649.0,
     'Long_PA': 19.0,
     'Long_PC': 15.0,
     'Medium_PA': 26.0,
     'Medium_PC': 26.0,
     'SPRINT': 5.0,
     'Short_PA': 2.0,
     'Short_PC': 2.0,
     'TS': 18.76,
     'TTP': 5796.0,
     'Total_PA': 47.0,
     'Total_PC': 43.0,
     'Total_PP': 0.91,
     'club': 'Brighton (ENG)',
     'name': 'Mathew RYAN',
     'num': 1,
     'position': 'GK'},
    {'DC': 9708.0,
     'DCIP': 3364.0,
     'DCNIP': 4153.0,
     'Long_PA': 10.0,
     'Long_PC': 7.0,
     'Medium_PA': 56.0,
     'Medium_PC': 55.0,
     'SPRINT': 27.0,
     'Short_PA': 4.0,
     'Short_PC': 3.0,
     'TA3RD': 0.08,
     'TOH': 0.16,
     'TPA': 0.02,
     'TS': 29.92,
     'TTP': 5796.0,
     'Total_PA': 70.0,
     'Total_PC': 65.0,
     'Total_PP': 0.93,
     'club': 'Al Ahli (KSA)',
     'name': 'Mark MILLIGAN',
     'num': 5,
     'position': 'DF'},

In [18]:
list(match_collection.aggregate([
    {'$group': {
        '_id': '$home_team.country',
        'home_goal': {'$sum' : '$home_team.score'}
    }},
    {'$sort': {'home_goal': -1}}
]))

[{'_id': 'Belgium', 'home_goal': 13},
 {'_id': 'France', 'home_goal': 12},
 {'_id': 'Russia', 'home_goal': 10},
 {'_id': 'England', 'home_goal': 6},
 {'_id': 'Brazil', 'home_goal': 6},
 {'_id': 'Uruguay', 'home_goal': 6},
 {'_id': 'Croatia', 'home_goal': 5},
 {'_id': 'Portugal', 'home_goal': 4},
 {'_id': 'Korea Republic', 'home_goal': 3},
 {'_id': 'Nigeria', 'home_goal': 3},
 {'_id': 'Spain', 'home_goal': 3},
 {'_id': 'Switzerland', 'home_goal': 2},
 {'_id': 'Colombia', 'home_goal': 2},
 {'_id': 'Saudi Arabia', 'home_goal': 2},
 {'_id': 'Japan', 'home_goal': 2},
 {'_id': 'Germany', 'home_goal': 2},
 {'_id': 'Sweden', 'home_goal': 2},
 {'_id': 'Serbia', 'home_goal': 1},
 {'_id': 'Panama', 'home_goal': 1},
 {'_id': 'IR Iran', 'home_goal': 1},
 {'_id': 'Poland', 'home_goal': 1},
 {'_id': 'Iceland', 'home_goal': 1},
 {'_id': 'Tunisia', 'home_goal': 1},
 {'_id': 'Argentina', 'home_goal': 1},
 {'_id': 'Denmark', 'home_goal': 1},
 {'_id': 'Senegal', 'home_goal': 0},
 {'_id': 'Australia', 'hom

In [19]:
list(match_collection.aggregate([
    {'$match' : {'$or': [{'home_team.country':'France'}, {'away_team.country':'Franceh'}] } },
    {'$group': {
        '_id': '$home_team.country',
        'home_goal': {'$sum' : '$home_team.score'}
    }}
]))

[{'_id': 'France', 'home_goal': 12}]